In [4]:
from concurrent.futures import ProcessPoolExecutor
from sharedConfig import generate_batches, BULK, BULK_PURITY, PURITY
from logBatchesNew import load_completed_batches, mark_batch_completed
import scaden
print(scaden.__file__)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
from scaden.simulate import simulation
from scaden.process import processing
from scaden.train import training
from scaden.predict import prediction

import logging
import tensorflow as tf
from anndata import read_h5ad
from scaden.model.functions import get_signature_genes, preprocess_h5ad_data, custom_preprocess_h5ad_data
logger = logging.getLogger(__name__)


import os, shutil, gc


import os
import itertools
import random

BASE_PATH = "/Users/nv4/gitClones/01_Deconvolution/03_Deconvolution_CTVT/033_Results/03_Deconvolution_CTVT_customScaden/snCountsCib10/0_scadenInput/"
BULK = os.path.join("/Users/nv4/gitClones/01_Deconvolution/03_Deconvolution_CTVT/033_Results/03_Deconvolution_CTVT_customScaden/0_scadenInput/03_Deconvolution_CTVT_customScaden_bulkCountsFinal.txt")
# BULK_PURITY = os.path.join("/Users/nv4/gitClones/01_Deconvolution/03_Deconvolution_CTVT/033_Results/03_Deconvolution_CTVT_customScaden/0_scadenInput/03_Deconvolution_CTVT_customScaden_bulkCountsGoiPurityNoBadAndy.txt")
BULK_PURITY = os.path.join("/Users/nv4/gitClones/01_Deconvolution/03_Deconvolution_CTVT/033_Results/03_Deconvolution_CTVT_customScaden/0_scadenInput/03_Deconvolution_CTVT_customScaden_bulkCountsFinal.txt")
# PURITY = "CTVT"
PURITY = None

/Users/nv4/gitClones/scadenPuritiesFullyRandomSim/scaden/__init__.py


In [5]:
subDir, bal, thr, ran, var, sam, cel, seed, lr, steps, cran = "/Users/nv4/gitClones/01_Deconvolution/03_Deconvolution_CTVT/033_Results/03_Deconvolution_CTVT_customScaden/snCountsCib50/0_scadenInput/snCountsCib50/", False, 0, 1, 0, 20, 500, 123, 0.01, 10,  0
pfix = f"{os.path.basename(subDir.rstrip('/'))}_bal{bal}_thresh{thr}_ran{ran}_var{var}_samp{sam}_cells{cel}_seed{seed}_lr{lr}_steps{steps}_cran{cran}_test"
simDir = f"/Users/nv4/gitClones/01_Deconvolution/03_Deconvolution_CTVT/033_Results/03_Deconvolution_CTVT_customScaden/1_simulate/{pfix}/"
  

In [6]:
  
os.makedirs(simDir, exist_ok=True)
simulation(
        simulate_dir=simDir,
        data_dir=subDir,
        sample_size=cel,
        num_samples=sam,
        pattern="*_counts.txt",
        unknown_celltypes=["unknown"],
        out_prefix=pfix,
        fmt="txt",
        balance=bal,
        threshold=thr,
        percRandom=ran,
        saveProp=True,
        seed=seed,
        remMerged=True,
        purity = PURITY, 
    cran=1
    )


--------------------------------------------------------------------
----------------------------- SIMULATION ---------------------------
--------------------------------------------------------------------
Generating 20 samples:
  - 20 with random cell type proportions:
    - 20 mixed
    - 0 single-sample
    --> 0 single-sample per id
  - 0 with original proportions:
    - 0 mixed
    - 0 single-sample

    --> 0 single-sample per id
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
Works.
--------------------------------------------------------------------
--------------------------------------------------------------------
--------------------------------------------------------------------
Simulated dataset. Saved as snCountsCib50.h5ad
Saved simulated proportions and counts.
Merged and saved dataset. Saved as /Users/nv4/gitClones/01_Deconvolution/03_Deconvolution_CTVT/033_Results/03_Deconvolution_C

/Users/nv4/miniforge3/envs/scadenPuritiesFullyRandomEnvX86/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [7]:
training_data = os.path.join(simDir, pfix + ".h5ad")
# processed_path = os.path.join(procDir, pfix + "_Processed.h5ad")
 


In [8]:
# Get the common genes (signature genes)
raw_input = read_h5ad(training_data)
sig_genes_complete = list(raw_input.var_names)
print(f"Sig_genes_complete: {len(sig_genes_complete)}")
print(sig_genes_complete)


Sig_genes_complete: 40
['ENSCAFG00000000865', 'ENSCAFG00000001519', 'ENSCAFG00000002065', 'ENSCAFG00000002357', 'ENSCAFG00000003426', 'ENSCAFG00000004043', 'ENSCAFG00000006069', 'ENSCAFG00000006766', 'ENSCAFG00000007673', 'ENSCAFG00000007934', 'ENSCAFG00000009156', 'ENSCAFG00000010381', 'ENSCAFG00000010387', 'ENSCAFG00000011726', 'ENSCAFG00000011740', 'ENSCAFG00000011751', 'ENSCAFG00000012780', 'ENSCAFG00000012802', 'ENSCAFG00000012876', 'ENSCAFG00000012880', 'ENSCAFG00000013773', 'ENSCAFG00000013933', 'ENSCAFG00000013968', 'ENSCAFG00000014213', 'ENSCAFG00000014837', 'ENSCAFG00000014920', 'ENSCAFG00000016071', 'ENSCAFG00000016429', 'ENSCAFG00000017455', 'ENSCAFG00000018219', 'ENSCAFG00000020027', 'ENSCAFG00000020110', 'ENSCAFG00000024903', 'ENSCAFG00000028568', 'ENSCAFG00000028574', 'ENSCAFG00000029321', 'ENSCAFG00000030180', 'ENSCAFG00000030305', 'ENSCAFG00000031753', 'ENSCAFG00000032358']


In [9]:
sig_genes = get_signature_genes(input_path=BULK, 
                                    sig_genes_complete=sig_genes_complete, 
                                    var_cutoff=0, 
                                    ignore_genes=None)
print(f"Sig_genes: {len(sig_genes)}")

Sig_genes: 40


In [10]:
import pandas as pd
# def get_signature_genes(input_path, sig_genes_complete, var_cutoff=0.1, ignore_genes = None):
#     """
#     Get overlap between signature genes and available genes
#     :param input_path:
#     :param sig_genes_complete:
#     :return: new sig_genes
#     """
data = pd.read_table(BULK, index_col=0)
print(data)

                    3145T11a  3145T11a_n  3145T1c_p  3145T1c_n  3203T1A2b_2  \
ENSCAFG00000000001        82         941        126        829         1151   
ENSCAFG00000000002         1          14          0          8           14   
ENSCAFG00000000005        19          32         95          9           24   
ENSCAFG00000000007       102        3243        951       3690         3111   
ENSCAFG00000000008        73         674       1985        652         2217   
...                      ...         ...        ...        ...          ...   
ENSCAFG00000050066         0           0          0          0            0   
ENSCAFG00000050068         2           1          0          5            0   
ENSCAFG00000050077         0           0          0          0            0   
ENSCAFG00000050078       794        2087       7689       2115        13345   
ENSCAFG00000050081         0           0          0          0            0   

                    3203T1A2b_n  3203T1A2c_2  3203T

In [7]:
keep = data.var(axis=1) > 0.1
print(keep)

ENSCAFG00000030180    True
ENSCAFG00000032342    True
ENSCAFG00000012802    True
ENSCAFG00000014213    True
ENSCAFG00000004496    True
ENSCAFG00000020340    True
ENSCAFG00000014203    True
ENSCAFG00000002357    True
ENSCAFG00000000803    True
ENSCAFG00000000812    True
ENSCAFG00000000832    True
ENSCAFG00000000848    True
ENSCAFG00000032222    True
ENSCAFG00000004043    True
ENSCAFG00000002955    True
ENSCAFG00000000426    True
ENSCAFG00000007464    True
ENSCAFG00000008612    True
ENSCAFG00000002065    True
ENSCAFG00000028568    True
ENSCAFG00000003133    True
ENSCAFG00000006766    True
ENSCAFG00000003169    True
ENSCAFG00000004970    True
ENSCAFG00000012226    True
ENSCAFG00000010352    True
ENSCAFG00000004782    True
ENSCAFG00000041713    True
ENSCAFG00000015228    True
ENSCAFG00000013633    True
ENSCAFG00000010676    True
ENSCAFG00000011751    True
ENSCAFG00000010858    True
ENSCAFG00000010997    True
ENSCAFG00000012876    True
ENSCAFG00000012880    True
ENSCAFG00000010387    True
E

In [14]:
data = data.loc[keep]
print(data)

                    3145T11a  3145T11a_n  3145T1c_p  3145T1c_n  3203T1A2b_2  \
ENSCAFG00000030180       138         423       1985        247         1573   
ENSCAFG00000032342        15         459         54         57          393   
ENSCAFG00000012802         6         101         48         10          159   
ENSCAFG00000014213         7          27        104         28          146   
ENSCAFG00000004496       320        8890       1838       5738        12250   
ENSCAFG00000020340        14        1659         33        185          184   
ENSCAFG00000014203        39         420        731        498         1297   
ENSCAFG00000002357         0          19          0          6           11   
ENSCAFG00000000803       923       11803      35539       5176        24220   
ENSCAFG00000000812       419        8752       6438       4489        21551   
ENSCAFG00000000832        84        4407       1413       1436         4789   
ENSCAFG00000000848       156        1536       3176 

In [12]:
available_genes = list(data.index)
available_genes

['ENSCAFG00000000001',
 'ENSCAFG00000000002',
 'ENSCAFG00000000005',
 'ENSCAFG00000000007',
 'ENSCAFG00000000008',
 'ENSCAFG00000000009',
 'ENSCAFG00000000010',
 'ENSCAFG00000000011',
 'ENSCAFG00000000012',
 'ENSCAFG00000000013',
 'ENSCAFG00000000014',
 'ENSCAFG00000000015',
 'ENSCAFG00000000016',
 'ENSCAFG00000000017',
 'ENSCAFG00000000019',
 'ENSCAFG00000000020',
 'ENSCAFG00000000021',
 'ENSCAFG00000000022',
 'ENSCAFG00000000024',
 'ENSCAFG00000000025',
 'ENSCAFG00000000026',
 'ENSCAFG00000000027',
 'ENSCAFG00000000028',
 'ENSCAFG00000000029',
 'ENSCAFG00000000030',
 'ENSCAFG00000000031',
 'ENSCAFG00000000032',
 'ENSCAFG00000000034',
 'ENSCAFG00000000036',
 'ENSCAFG00000000037',
 'ENSCAFG00000000038',
 'ENSCAFG00000000039',
 'ENSCAFG00000000040',
 'ENSCAFG00000000042',
 'ENSCAFG00000000043',
 'ENSCAFG00000000044',
 'ENSCAFG00000000045',
 'ENSCAFG00000000049',
 'ENSCAFG00000000052',
 'ENSCAFG00000000054',
 'ENSCAFG00000000056',
 'ENSCAFG00000000058',
 'ENSCAFG00000000059',
 'ENSCAFG00

In [13]:
print(available_genes)
print(sig_genes_complete)


['ENSCAFG00000000001', 'ENSCAFG00000000002', 'ENSCAFG00000000005', 'ENSCAFG00000000007', 'ENSCAFG00000000008', 'ENSCAFG00000000009', 'ENSCAFG00000000010', 'ENSCAFG00000000011', 'ENSCAFG00000000012', 'ENSCAFG00000000013', 'ENSCAFG00000000014', 'ENSCAFG00000000015', 'ENSCAFG00000000016', 'ENSCAFG00000000017', 'ENSCAFG00000000019', 'ENSCAFG00000000020', 'ENSCAFG00000000021', 'ENSCAFG00000000022', 'ENSCAFG00000000024', 'ENSCAFG00000000025', 'ENSCAFG00000000026', 'ENSCAFG00000000027', 'ENSCAFG00000000028', 'ENSCAFG00000000029', 'ENSCAFG00000000030', 'ENSCAFG00000000031', 'ENSCAFG00000000032', 'ENSCAFG00000000034', 'ENSCAFG00000000036', 'ENSCAFG00000000037', 'ENSCAFG00000000038', 'ENSCAFG00000000039', 'ENSCAFG00000000040', 'ENSCAFG00000000042', 'ENSCAFG00000000043', 'ENSCAFG00000000044', 'ENSCAFG00000000045', 'ENSCAFG00000000049', 'ENSCAFG00000000052', 'ENSCAFG00000000054', 'ENSCAFG00000000056', 'ENSCAFG00000000058', 'ENSCAFG00000000059', 'ENSCAFG00000000060', 'ENSCAFG00000000061', 'ENSCAFG0

In [14]:

new_sig_genes = list(set(available_genes).intersection(sig_genes_complete))
print(new_sig_genes)

['ENSCAFG00000009156', 'ENSCAFG00000002357', 'ENSCAFG00000012802', 'ENSCAFG00000013968', 'ENSCAFG00000010381', 'ENSCAFG00000014837', 'ENSCAFG00000016071', 'ENSCAFG00000014213', 'ENSCAFG00000028574', 'ENSCAFG00000031753', 'ENSCAFG00000013773', 'ENSCAFG00000000865', 'ENSCAFG00000016429', 'ENSCAFG00000011726', 'ENSCAFG00000006069', 'ENSCAFG00000012780', 'ENSCAFG00000007934', 'ENSCAFG00000024903', 'ENSCAFG00000028568', 'ENSCAFG00000014920', 'ENSCAFG00000018219', 'ENSCAFG00000017455', 'ENSCAFG00000006766', 'ENSCAFG00000029321', 'ENSCAFG00000030180', 'ENSCAFG00000030305', 'ENSCAFG00000004043', 'ENSCAFG00000011740', 'ENSCAFG00000001519', 'ENSCAFG00000010387', 'ENSCAFG00000032358', 'ENSCAFG00000007673', 'ENSCAFG00000002065', 'ENSCAFG00000013933', 'ENSCAFG00000012876', 'ENSCAFG00000020110', 'ENSCAFG00000003426', 'ENSCAFG00000020027', 'ENSCAFG00000011751', 'ENSCAFG00000012880']


In [15]:

n_sig_genes = len(new_sig_genes)
logger.info(f"Found [cyan]{n_sig_genes}[/cyan] common genes.")
print(new_sig_genes)
print(len(new_sig_genes))


['ENSCAFG00000009156', 'ENSCAFG00000002357', 'ENSCAFG00000012802', 'ENSCAFG00000013968', 'ENSCAFG00000010381', 'ENSCAFG00000014837', 'ENSCAFG00000016071', 'ENSCAFG00000014213', 'ENSCAFG00000028574', 'ENSCAFG00000031753', 'ENSCAFG00000013773', 'ENSCAFG00000000865', 'ENSCAFG00000016429', 'ENSCAFG00000011726', 'ENSCAFG00000006069', 'ENSCAFG00000012780', 'ENSCAFG00000007934', 'ENSCAFG00000024903', 'ENSCAFG00000028568', 'ENSCAFG00000014920', 'ENSCAFG00000018219', 'ENSCAFG00000017455', 'ENSCAFG00000006766', 'ENSCAFG00000029321', 'ENSCAFG00000030180', 'ENSCAFG00000030305', 'ENSCAFG00000004043', 'ENSCAFG00000011740', 'ENSCAFG00000001519', 'ENSCAFG00000010387', 'ENSCAFG00000032358', 'ENSCAFG00000007673', 'ENSCAFG00000002065', 'ENSCAFG00000013933', 'ENSCAFG00000012876', 'ENSCAFG00000020110', 'ENSCAFG00000003426', 'ENSCAFG00000020027', 'ENSCAFG00000011751', 'ENSCAFG00000012880']
40
